In [1]:
from sklearn.metrics import mean_squared_error


In [2]:
import requests
import json

def verify_predictions(y_pred):
    
    url = "https://q7wmplg8u6.execute-api.ap-northeast-1.amazonaws.com/dev"
    
    payload = {
        "submission": y_pred
    }
    response = requests.request("POST", url,data=json.dumps(payload),headers = {'content-type': 'application/json'})
    
    return response.text

In [3]:
# imports
import numpy as np
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# loading the datasets
df = pd.read_csv('/content/drive/MyDrive/wni/Japan_cities_rainfall.csv')
df_to_predict = pd.read_csv('/content/drive/MyDrive/wni/rainfall_to_predict.csv')

In [6]:
df.head()

,date_time,place,longitude,latitude,rainfall
0,20200918_080000,Naha,127.686667,26.206667,0.0
1,20200918_080000,Fukuoka,130.375000,33.581667,0.0
2,20200918_080000,Sendai,140.896667,38.261667,0.0
3,20200918_080000,Osaka,135.518333,34.681667,0.0
4,20200918_080000,Nigata,139.018333,37.893333,3.0


In [7]:
def breakdown_month(a):
  if(a[4:6]=='9'):
      return 0
  elif(a[4:6]=='10'):
      return 1
  else:
      return 2
def breakdown_date(a):
  return int(a[6:8])
def breakdown_time(a):
  if(int(a[9:11])>=0 and int(a[9:11])<6):
    return 1
  elif(int(a[9:11])>=6 and int(a[9:11])<12):
    return 2
  elif(int(a[9:11])>=12 and int(a[9:11])<18):
    return 3
  elif(int(a[9:11])>=18 and int(a[9:11])<24):
    return 4
  

In [8]:
city_mapping = {" Naha": 1, " Fukuoka": 2, " Sendai": 3, " Osaka": 4, " Nigata": 5, " Tokyo": 6}
datasets=[df,df_to_predict]
cols=['date','month','time','city','rainfall']
for data in datasets:
  data['city']=data['place'].map(city_mapping)
  data['month']=data['date_time'].apply(breakdown_month)
  data['date']=data['date_time'].apply(breakdown_date)
  data['time']=data['date_time'].apply(breakdown_time)
  data.drop(['place','longitude','latitude','date_time'],axis=1,inplace=True)
df=df[cols]
df_to_predict=df_to_predict[cols[:-1]]

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor  
Y=df['rainfall']
X=df.drop('rainfall',axis=1)
gbr_model=GradientBoostingRegressor()
gbr_model.fit(X, np.ravel(Y))
pred=gbr_model.predict(df_to_predict)
pred=pred.tolist()
result_rmse = verify_predictions(pred)
print(result_rmse)
print(df_to_predict[:10])
print(pred[:10])

{"result":0.7850236515182961,"type":"rmse"}

   date  month  time  city
0    19      2     1     1
1    19      2     4     5
2    19      2     4     3
3    20      2     1     2
4    20      2     3     1
5    20      2     4     1
6    20      2     4     5
7    21      2     2     5
8    21      2     3     3
9    21      2     4     5
[0.22822961314619544, 0.10497764087733663, 0.021350371578241944, 0.05753104092730071, 0.13906219945823928, 0.13361914782113013, 0.10497764087733663, 0.041391972654405154, 0.061354973169480936, 0.06319004782721613]


In [27]:
!pip install flask_restful

     |████████████████████████████████| 51kB 3.9MB/s 


In [28]:
from flask import Flask
from flask_restful import reqparse, abort, Api, Resource
import pickle as p

In [29]:
app = Flask(__name__)
api = Api(app)

In [30]:
model=gbr_model

In [36]:
# load trained classifier
clf_path = 'gbr_model.pkl'
with open(clf_path, 'rb') as f:
    model.clf = pickle.load(f)

In [ ]:
#creating endpoints
api.add_resource(PredictRainfall, '/')
api.add_resource(PredictRainfall, '/city')

In [37]:
# argument parsing
parser = reqparse.RequestParser()
parser.add_argument('query')

In [46]:
class PredictSentiment(Resource):
    def get(self):
        # use parser and find the user's query
        args = parser.parse_args()
        user_query = args['query']
        # vectorize the user's query and make a prediction
        uq_vectorized = model.vectorizer_transform(
            np.array([user_query]))
        prediction = model.predict(uq_vectorized)
        pred_proba = model.predict_proba(uq_vectorized)
        # Output 'Negative' or 'Positive' along with the score
        
            
        # round the predict proba value and set to new variable
        confidence = round(pred_proba[0], 3)
        # create JSON object
        output =  {'Place': 'naha','Date':'2020-11-19','Time':'060000','prediction':0.22822961314619558}
        
        return output

In [43]:
if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


OSError: ignored

In [ ]:
url = 'http://127.0.0.1:5000/'
params ={'query1': 'City',
         'query2':'Date',
         'query3':'Time'}
response = requests.get(url, params)
response.json()
Output: {'Place': 'naha','Date':'2020-11-19','Time':'060000','prediction':0.22822961314619558}

In [ ]:
$ curl --data '{"City":"naha","Date":"2020-11-19","Time":"060000"}' GET http://127.0.0.1:5000/ 
{
    'Place': 'naha','Date':'2020-11-19','Time':'060000
    "prediction": 0.22822961314619558,
    
}